In [32]:
# import all necessary packages

from urllib2 import urlopen
import json
import pandas as pd
import elasticsearch
import requests
import indicoio
from sklearn.cluster import MeanShift
from nltk.corpus import stopwords
indicoio.config.api_key = '9369f9b0a0d9333eeb47aa93f44ffe0f'
import pprint
import re
import numpy as np
from sklearn.cluster import KMeans
import ast
pp = pprint.PrettyPrinter(indent=4)
TAG_RE = re.compile(r'<[^>]+>')
def cleanstring(x):
    re.sub('[^a-zA-Z0-9-_*.]','', x)
def remove_tags(text):
    return TAG_RE.sub('', text)
from HTMLParser import HTMLParser
class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

# import stopword

stopset = set(stopwords.words('english'))
stopset.update(["em", "strong", 'wp', 'com', '8217', "jpeg", "http", "uploads", "new", "span", "family", "font", "arial", 
                "game", "style", "color", "twitter", "monothetic", "id", "div", "class", "text", "input", "re", "type", "src",
               "value", "data", "maxlength", "levels", "facebook", "first", "test", "wmode", "ifr_source", "h4", "attr", "function"
               , "video", "img", "_blank", "https", "content", "target", "href", "www", "000000", "000080", "ul li", "en us", "xbox",
               "01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "2015", "2016", "2017", "xbox one", "news", "alt", "srcset"
               , "youtube", "one", "jpg", "height", "en", "wire", "px", "11", "pt", "xbox wire", "xbox live", "microsoft", "game"
               , "games"])

In [34]:
# start an instance of elasticsearch
es = elasticsearch.Elasticsearch([{'host': 'localhost', 'port': 9200}])

# create the mappings of elasticsearch for the webhose articles

mapping = {
    "articles": {
        "properties": {
            "article_title": {"type": "string", "similarity": "default"},
            "link": {"type": "string"},
            "content": {"type": "string", "similarity": "default"},
            "author": {"type": "string"}
        }
    }
}

# Create a new "xbwire" index that includes "articles" with the above mapping
es.indices.create("xbwire5")
es.indices.put_mapping(index="xbwire5", doc_type="articles", body=mapping)

{u'acknowledged': True}

In [35]:
# get the Xbox Wire articles, break them into the title, link, body, and author 

# store them within articles_coll

articles_coll = []

page_num = 1
while page_num < 3:
    str_pn = str(page_num)
    json_url = 'https://news.xbox.com/wp-json/posts' + '?page=' + str_pn
    open_url = urlopen(json_url)
    data = json.load(open_url)
    for x in data:
        articles = x['title'].encode('utf-8'), x['link'], strip_tags(x['content'].encode('utf-8').lower()), x['author']['name']
        articles_coll.append(articles)
    page_num += 1
    
first_url_string = 'https://webhose.io/search?token=89194856-ed59-4c4b-8037-758625bf7327&format=json&q=xbox%20language%3A(english)%20thread.country%3AUS%20site%3A'
end_url_string = '%20(site_type%3Anews%20OR%20site_type%3Ablogs)'
link_excel = pd.read_excel('Publication with Links.xlsx')

publications = pd.read_excel('Publication with Links.xlsx')

urls = ['gamespot.com', 'ign.com', 'kotaku.com', 'theverge.com', 'polygon.com', 'engadget.com', 'gizmodo.com', 'destructoid.com', 'cnet.com']

webhose_articles = []

# get the webhose articles

for x in urls:
    r = urlopen(first_url_string + x + end_url_string)
    data = json.load(r)
    for x in data['posts']:
        articles = x['title'].encode('utf-8').lower(), x['url'], strip_tags(x['text'].encode('utf-8').lower()), x['author']
        webhose_articles.append(articles)

In [36]:
# Using Indico.io, make a set of keywords and entities, and pass it back to the articles_coll so that everything travels together

new_articles_coll = []
for x in articles_coll:
    body = x[2]
    xw_kw_list = []
    if len(body) > 0:
        dict_entities = indicoio.keywords(body, top_n=15, version=2, relative=True, threshold=0.1)
        for y in dict_entities.keys():
            if not y in stopset:
                xw_kw_list.append((y, "keyword"))
        people_entities = indicoio.people(body, threshold=0.5)
        for y in people_entities:
            if not y['text'] in stopset:
                people_tuple = y['text'], "entity"
                xw_kw_list.append(people_tuple)
        places_entities = indicoio.places(body, threshold=0.5)
        for y in places_entities:
            if not y['text'] in stopset:
                places_tuple = y['text'], "entity"
                xw_kw_list.append(places_tuple)
        org_entities = indicoio.organizations(body, threshold=0.5)
        for y in org_entities:
            if not y['text'] in stopset:
                org_tuple = y['text'], "entity"
                xw_kw_list.append(org_tuple)
    else:
        pass
    xw_kw_list.append(x[3].split(","))
    to_append = x[0], x[1], x[2], x[3], xw_kw_list
    new_articles_coll.append(to_append)

In [37]:
# index the webhose articles into an instance of elasticsearch to run queries against them

i = 0
for x in webhose_articles:
    try:
        es.index(index='xbwire5', doc_type='articles', id=i, body={"article_title": x[0], "link": x[1], "content": x[2], "author": x[3]})
    except:
        pass
    i += 1
    
print i

141


In [43]:
# Using the keywords from the Indico.io algorithm, build queries to feed into elasticsearch, feed the queries back
# a list with its elements to have them all travel together again

articles_with_query = []

for x in new_articles_coll:
    queries = []
    for y in x[4]:
        if len(y) > 1:
            word_type = y[1]
            word = y[0]
            if word_type == "keyword":
                lower_title = x[0].lower()
                formatted_keyword = word.encode('utf-8')
                if word_type.find('\n') > 0:
                    pass
                elif len(y) == 0:
                    pass
                elif formatted_keyword == " ":
                    pass
                elif formatted_keyword.lower() in lower_title:
                    title_word = '{"multi_match": ' + "{" + '"query": ' + '"' + formatted_keyword + '", ' + '"type": ' + '"best_fields", ' + '"fields": ' + '[ "article_title^3", "content" ], ' + '"tie_breaker": ' + "0.3, " + '"minimum_should_match": ' + '"40%"' +"}}, "
                    queries.append(title_word)
                elif " " in formatted_keyword:
                    phrase = formatted_keyword.split(" ")
                    for new_word in phrase:
                        if new_word in lower_title:
                            partial_word = '{"multi_match": ' + "{" + '"query": ' + '"' + new_word + '", ' + '"type": ' + '"best_fields", ' + '"fields": ' + '[ "article_title^3", "content" ], ' + '"tie_breaker": ' + "0.3, " + '"minimum_should_match": ' + '"40%"' +"}}, "
                            queries.append(partial_word)
                else:
                    try:
                        word = '{"multi_match": ' + "{" + '"query": ' + '"' + formatted_keyword + '", ' + '"type": ' + '"best_fields", ' + '"fields": ' + '[ "article_title", "content" ], ' + '"tie_breaker": ' + "0.3, " + '"minimum_should_match": ' + '"40%"' +"}}, "
                        queries.append(word)
                    except:
                        pass
            elif word_type == "entity":
                formatted_keyword = word.encode('utf-8')
                if word.find('\n') > 0:
                    pass
                elif len(y) == 0:
                    pass
                elif formatted_keyword == " ":
                    pass
                elif formatted_keyword.lower() in lower_title:
                    title_word = '{"multi_match": ' + "{" + '"query": ' + '"' + formatted_keyword + '", ' + '"type": ' + '"best_fields", ' + '"fields": ' + '[ "article_title^3", "content^3" ], ' + '"tie_breaker": ' + "0.3, " + '"minimum_should_match": ' + '"30%"' +"}}, "
                    queries.append(title_word)
                elif " " in formatted_keyword:
                    phrase = formatted_keyword.split(" ")
                    for new_word in phrase:
                        if new_word in lower_title:
                            partial_word = '{"multi_match": ' + "{" + '"query": ' + '"' + new_word + '", ' + '"type": ' + '"best_fields", ' + '"fields": ' + '[ "article_title^3", "content^3" ], ' + '"tie_breaker": ' + "0.3, " + '"minimum_should_match": ' + '"30%"' +"}}, "
                            queries.append(partial_word)
                        else:
                            pass
                else:
                    try:
                        word = '{"multi_match": ' + "{" + '"query": ' + '"' + formatted_keyword + '", ' + '"type": ' + '"best_fields", ' + '"fields": ' + '[ "article_title^2", "content^3" ], ' + '"tie_breaker": ' + "0.3, " + '"minimum_should_match": ' + '"30%"' +"}}, "
                        queries.append(word)
                    except:
                        pass
    set_queries = set(queries)
    htw = ''.join(set_queries)
    length = len(htw)
    format_htw = "[" + htw[:length-2] + "]"
    to_add = [x[0], x[1], x[2], format_htw]
    articles_with_query.append(to_add)

In [54]:
# Article by article, query by query, gather results collected from elasticsearch and store them together
# After that, use clustering and thresholds to combine the most relevant articles together that are most likely to be based off of
# the Xbox Wire article

Titles = []
Article_Link = []
Article_Relevance_Label = []
Rlt_Article_Links = []    
Article_Relevance_Score = []
Article_Title =[]
n = 1
while n < 2:
    for x in articles_with_query:
        query = x[3]
        if len(query) > 0:
            searches = es.search(index="xbwire5", body={"query": {"bool": {"should": ast.literal_eval(query)}}})
            relevance_score = []
            other_index = []
            article_name = []
            rlted_article_lk = []
        for y in searches['hits']['hits']:
            if y['_score'] < 1 and y['_score'] > 0.001:
                relevance = y['_score']
                relevance_score.append(relevance)
                other_index.append(1)
                article_name.append(y['_source']['article_title'])
                rlted_article_lk.append(y['_source']['link'])
            else:
                pass
        if len(relevance_score) > 3:
            X = np.column_stack((relevance_score, other_index))
            ms = MeanShift()
            ms.fit(X)
            labels = ms.labels_
            cluster_centers = ms.cluster_centers_
            n_clusters_ = len(np.unique(labels))
            for i in range(len(X)):
                if labels[i] != 0:
                    Titles.append(x[0])
                    Article_Link.append(x[1])
                    Article_Title.append(article_name[i])
                    Rlt_Article_Links.append(rlted_article_lk[i])
                    Article_Relevance_Score.append(X[i])
                    Article_Relevance_Label.append(labels[i])
        else:
            pass

        n += 1

In [55]:
# Use pandas to easily everything to a CSV document

results = pd.DataFrame(columns=['Xbox Wire Title', 'Article Link', 'Related Article Title', 'Related Article Label', 'Related Article Link', 'Related Article Relevance'])

results['Xbox Wire Title'] = Titles
results['Article Link'] = Article_Link
results['Related Article Title'] = Article_Title
results['Related Article Link'] = Rlt_Article_Links
results['Related Article Label'] = Article_Relevance_Label
results['Related Article Relevance'] = Article_Relevance_Score

results.to_csv("Article_Results.csv", index=False)